In [2]:
from cot import Collection
import pandas as pd
import numpy as np
import krippendorff
from scipy import stats
pd.set_option('display.max_rows', 200)

In [3]:
coll = Collection.from_json("/home/kon/work/ThoughtSource/libs/cot/cot/datasets/thoughtsource/thoughtsource_33_paper.json")

[nltk_data] Downloading package punkt to /home/kon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
ev = coll.evaluate(overwrite=True)

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

In [5]:
df = coll.collection_to_dataframe()

In [6]:
df.head(3)

,dataset,split,id,type,number_choices,answer_label,prompt,instruction,cot_trigger,answer_from_choices,correct_answer,model
0,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,multiplechoice,5,E,None_None,None,None,E,True,command-xlarge-nightly
1,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,multiplechoice,5,E,None_None,None,None,E,True,text-davinci-003
2,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,multiplechoice,5,E,None_None,None,None,E,True,flan-T5-xxl


In [7]:
df.drop(columns=["number_choices"], inplace=True)

In [8]:
table = df.groupby(["model", "prompt"]).mean("correct_answer").pivot_table(index="model", columns="prompt", values="correct_answer")
table

prompt,None_None,None_kojima-01,None_zhou-01,qa-10_None,qa-12_None,qa-13_None,qa-16_None,qa-17_None,refl-01_None,zhou-01-ins_None
model,,,,,,,,,,
command-xlarge-nightly,0.449495,0.424242,0.505051,0.444444,0.459596,0.484848,0.515152,0.469697,0.434343,0.474747
flan-T5-xxl,0.621212,0.621212,0.575758,0.601010,0.631313,0.611111,0.540404,0.611111,0.570707,0.590909
gpt-3.5-turbo,0.712121,0.686869,0.696970,0.666667,0.666667,0.656566,0.651515,0.681818,0.651515,0.671717
gpt-4,0.792929,0.833333,0.863636,0.833333,0.797980,0.818182,0.797980,0.777778,0.797980,0.813131
text-davinci-002,0.565657,0.525253,0.611111,0.530303,0.580808,0.565657,0.611111,0.494949,0.540404,0.525253
text-davinci-003,0.595960,0.580808,0.631313,0.606061,0.636364,0.595960,0.555556,0.585859,0.611111,0.626263


In [9]:
table = df.groupby(["model", "dataset"]).mean("correct_answer").pivot_table(index="model", columns="dataset", values="correct_answer")
table

dataset,commonsense_qa,med_qa,medmc_qa,open_book_qa,strategy_qa,worldtree
model,,,,,,
command-xlarge-nightly,0.657576,0.263636,0.309091,0.572727,0.248485,0.745455
flan-T5-xxl,0.833333,0.212121,0.342424,0.766667,0.618182,0.812121
gpt-3.5-turbo,0.684848,0.503030,0.593939,0.757576,0.596970,0.909091
gpt-4,0.748485,0.642424,0.781818,0.912121,0.815152,0.975758
text-davinci-002,0.690909,0.272727,0.396970,0.560606,0.533333,0.875758
text-davinci-003,0.690909,0.339394,0.393939,0.693939,0.621212,0.875758


In [10]:
table = df.groupby(["prompt", "dataset"]).mean("correct_answer").pivot_table(index="prompt", columns="dataset", values="correct_answer")
table

dataset,commonsense_qa,med_qa,medmc_qa,open_book_qa,strategy_qa,worldtree
prompt,,,,,,
None_None,0.722222,0.368687,0.454545,0.727273,0.595960,0.868687
None_kojima-01,0.696970,0.373737,0.424242,0.681818,0.666667,0.828283
None_zhou-01,0.717172,0.393939,0.530303,0.777778,0.595960,0.868687
qa-10_None,0.752525,0.358586,0.474747,0.722222,0.520202,0.853535
qa-12_None,0.722222,0.383838,0.494949,0.737374,0.520202,0.914141
qa-13_None,0.742424,0.368687,0.469697,0.656566,0.606061,0.888889
qa-16_None,0.661616,0.398990,0.510101,0.661616,0.606061,0.833333
qa-17_None,0.646465,0.363636,0.484848,0.696970,0.590909,0.838384
refl-01_None,0.757576,0.353535,0.419192,0.722222,0.489899,0.863636


In [281]:
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (model, prompt), group in df.groupby(['model', 'prompt']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        resampled = group['correct_answer'].sample(n=len(group), replace=True)
        bootstrapped_means.append(resampled.mean())
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'model': model,
        'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)
mean_pivot = results_df.pivot_table(values='mean', index='model', columns='prompt')
ci_pivot_lower = results_df.pivot_table(values='ci_lower', index='model', columns='prompt')
ci_pivot_upper = results_df.pivot_table(values='ci_upper', index='model', columns='prompt')

In [ ]:
# every dataset seperately
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (model, prompt, dataset), group in df.groupby(['model', 'prompt', 'dataset']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        resampled = group['correct_answer'].sample(n=len(group), replace=True)
        bootstrapped_means.append(resampled.mean())
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        'model': model,
        'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)
# mean_pivot = results_df.pivot_table(values='mean', index='model', columns='prompt')
# ci_pivot_lower = results_df.pivot_table(values='ci_lower', index='model', columns='prompt')
# ci_pivot_upper = results_df.pivot_table(values='ci_upper', index='model', columns='prompt')

In [11]:
# KRIPPENDORFF ALPHA
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (model, prompt, dataset), group in df.groupby(['model', 'prompt', 'dataset']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        answer_label = group['answer_label'].sample(n=len(group), replace=True)
        indices = answer_label.index
        answer_from_choices = group.loc[indices, 'answer_from_choices']

        metric = krippendorff.alpha(reliability_data=[list(answer_label), list(answer_from_choices)],level_of_measurement='nominal')

        bootstrapped_means.append(metric)
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        'model': model,
        'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)
# mean_pivot = results_df.pivot_table(values='mean', index='model', columns='prompt')
# ci_pivot_lower = results_df.pivot_table(values='ci_lower', index='model', columns='prompt')
# ci_pivot_upper = results_df.pivot_table(values='ci_upper', index='model', columns='prompt')

In [30]:
filtered_df = results_df[(results_df['dataset'] == 'commonsense_qa') & (results_df['model'] == 'command-xlarge-nightly')]
filtered_df


,dataset,model,prompt,mean,ci_lower,ci_upper
0,commonsense_qa,command-xlarge-nightly,None_None,0.379762,0.167625,0.591751
6,commonsense_qa,command-xlarge-nightly,None_kojima-01,0.342267,0.117617,0.548108
12,commonsense_qa,command-xlarge-nightly,None_zhou-01,0.644224,0.444049,0.843281
18,commonsense_qa,command-xlarge-nightly,qa-10_None,0.610482,0.400533,0.805062
24,commonsense_qa,command-xlarge-nightly,qa-12_None,0.616859,0.400213,0.810719
30,commonsense_qa,command-xlarge-nightly,qa-13_None,0.688805,0.497202,0.877627
36,commonsense_qa,command-xlarge-nightly,qa-16_None,0.575817,0.367378,0.770895
42,commonsense_qa,command-xlarge-nightly,qa-17_None,0.501976,0.295127,0.703509
48,commonsense_qa,command-xlarge-nightly,refl-01_None,0.608241,0.394005,0.799259
54,commonsense_qa,command-xlarge-nightly,zhou-01-ins_None,0.654134,0.456700,0.841367


In [ ]:
filtered_df = results_df[(results_df['dataset'] == 'strategy_qa') & (results_df['model'] == 'command-xlarge-nightly')]
filtered_df


,dataset,model,prompt,mean,ci_lower,ci_upper
0,commonsense_qa,command-xlarge-nightly,None_None,0.379762,0.167625,0.591751
6,commonsense_qa,command-xlarge-nightly,None_kojima-01,0.342267,0.117617,0.548108
12,commonsense_qa,command-xlarge-nightly,None_zhou-01,0.644224,0.444049,0.843281
18,commonsense_qa,command-xlarge-nightly,qa-10_None,0.610482,0.400533,0.805062
24,commonsense_qa,command-xlarge-nightly,qa-12_None,0.616859,0.400213,0.810719
30,commonsense_qa,command-xlarge-nightly,qa-13_None,0.688805,0.497202,0.877627
36,commonsense_qa,command-xlarge-nightly,qa-16_None,0.575817,0.367378,0.770895
42,commonsense_qa,command-xlarge-nightly,qa-17_None,0.501976,0.295127,0.703509
48,commonsense_qa,command-xlarge-nightly,refl-01_None,0.608241,0.394005,0.799259
54,commonsense_qa,command-xlarge-nightly,zhou-01-ins_None,0.654134,0.456700,0.841367


In [31]:
### The problem is that I have to compute the difference of ci_lower to ci_mean first and then compute sqrt of the values squared.

In [14]:
df = results_df.copy()

In [17]:
results_df

,dataset,model,prompt,mean,ci_lower,ci_upper
0,commonsense_qa,command-xlarge-nightly,None_None,0.379762,0.167625,0.591751
1,med_qa,command-xlarge-nightly,None_None,-0.008886,-0.158063,0.152357
2,medmc_qa,command-xlarge-nightly,None_None,0.032149,-0.160714,0.251061
3,open_book_qa,command-xlarge-nightly,None_None,0.411786,0.178722,0.636001
4,strategy_qa,command-xlarge-nightly,None_None,1.000000,1.000000,1.000000
...,...,...,...,...,...,...
355,med_qa,text-davinci-003,zhou-01-ins_None,0.187988,0.006933,0.397459
356,medmc_qa,text-davinci-003,zhou-01-ins_None,0.111358,-0.101770,0.330338
357,open_book_qa,text-davinci-003,zhou-01-ins_None,0.747855,0.565523,0.917669
358,strategy_qa,text-davinci-003,zhou-01-ins_None,1.000000,1.000000,1.000000


In [19]:
def custom_function(x):
    squared_values = x**2
    return np.sqrt(squared_values.sum())

# Group the DataFrame by 'model', 'prompt', and 'dataset'
grouped_df = df.groupby(['model', 'prompt', 'dataset'])

# Apply the custom function to each group
custom_results = grouped_df.agg(custom_function)

In [23]:
df.groupby(['model', 'prompt']).agg(custom_function)

/tmp/ipykernel_6759/3233886831.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  df.groupby(['model', 'prompt']).agg(custom_function)


mean  ci_lower  ci_upper
model                  prompt                                        
command-xlarge-nightly None_None         1.241314  1.089474  1.528231
                       None_kojima-01    1.089987  0.827710  1.425526
                       None_zhou-01      1.091961  0.697572  1.536680
                       qa-10_None        1.012295  0.813503  1.350927
                       qa-12_None        1.079459  0.829556  1.447083
                       qa-13_None        1.195796  0.989189  1.511650
                       qa-16_None        1.089998  0.762856  1.523165
                       qa-17_None        0.909037  0.474838  1.392879
                       refl-01_None      1.061583  0.848915  1.387753
                       zhou-01-ins_None  1.173808  0.970207  1.495002
flan-T5-xxl            None_None         1.674424  1.457095  1.940163
                       None_kojima-01    1.649051  1.423904  1.897808
                       None_zhou-01      1.549017  1.215815  1.896926
                       qa-10_None        1.648143  1.427552  1.895562
                       qa-12_None        1.713612  1.494980  1.945492
                       qa-13_None        1.618413  1.380689  1.866871
                       qa-16_None        1.421127  1.221863  1.705495
                       qa-17_None        1.605361  1.370889  1.890703
                       refl-01_None      1.636359  1.350920  1.918262
                       zhou-01-ins_None  1.626880  1.408275  1.869233
gpt-3.5-turbo          None_None         1.739306  1.325418  2.125028
                       None_kojima-01    1.556716  1.119960  1.962951
                       None_zhou-01      1.620638  1.197891  2.037962
                       qa-10_None        1.475451  1.020739  1.906597
                       qa-12_None        1.550899  1.106139  1.974751
                       qa-13_None        1.570305  1.195914  1.958496
                       qa-16_None        1.480640  1.002305  1.944308
                       qa-17_None        1.626976  1.202983  2.030203
                       refl-01_None      1.470504  1.019652  1.913693
                       zhou-01-ins_None  1.545990  1.178499  1.909527
gpt-4                  None_None         1.944968  1.598623  2.240375
                       None_kojima-01    2.030896  1.753762  2.282689
                       None_zhou-01      2.090378  1.829292  2.324307
                       qa-10_None        2.011531  1.679062  2.272574
                       qa-12_None        1.882386  1.564116  2.198962
                       qa-13_None        1.998585  1.766875  2.236025
                       qa-16_None        1.935845  1.641368  2.227457
                       qa-17_None        1.880984  1.610620  2.147270
                       refl-01_None      1.906924  1.577620  2.215068
                       zhou-01-ins_None  1.936998  1.566946  2.262292
text-davinci-002       None_None         1.365005  0.980056  1.760428
                       None_kojima-01    1.284387  0.920053  1.683092
                       None_zhou-01      1.449094  1.076456  1.854100
                       qa-10_None        1.363100  1.010836  1.769689
                       qa-12_None        1.403015  1.070128  1.779926
                       qa-13_None        1.424398  1.081639  1.794640
                       qa-16_None        1.356822  0.953245  1.771314
                       qa-17_None        1.118038  0.735880  1.531888
                       refl-01_None      1.346799  0.953217  1.752739
                       zhou-01-ins_None  1.439003  1.148343  1.762589
text-davinci-003       None_None         1.528794  1.207971  1.852601
                       None_kojima-01    1.459057  1.127275  1.827988
                       None_zhou-01      1.602936  1.317066  1.921515
                       qa-10_None        1.532028  1.157423  1.891545
                       qa-12_None        1.647469  1.320008  1.946179
                       qa-13_None        1.510099  1.26

In [16]:
custom_results

mean  ci_lower  \
model                  prompt           dataset                              
command-xlarge-nightly None_None        commonsense_qa  0.379762  0.167625   
                                        med_qa          0.008886  0.158063   
                                        medmc_qa        0.032149  0.160714   
                                        open_book_qa    0.411786  0.178722   
                                        strategy_qa     1.000000  1.000000   
...                                                          ...       ...   
text-davinci-003       zhou-01-ins_None med_qa          0.187988  0.006933   
                                        medmc_qa        0.111358  0.101770   
                                        open_book_qa    0.747855  0.565523   
                                        strategy_qa     1.000000  1.000000   
                                        worldtree       0.829630  0.658074   

                                                        ci_upper  
model                  prompt           dataset                   
command-xlarge-nightly None_None        commonsense_qa  0.591751  
                                        med_qa          0.152357  
                                        medmc_qa        0.251061  
                                        open_book_qa    0.636001  
                                        strategy_qa     1.000000  
...                                                          ...  
text-davinci-003       zhou-01-ins_None med_qa          0.397459  
                                        medmc_qa        0.330338  
                                        open_book_qa    0.917669  
                                        strategy_qa     1.000000  
                                        worldtree       0.959452  

[360 rows x 3 columns]

In [ ]:
# Define bootstrapping parameters
n_bootstraps = 1000
confidence_level = 0.95
np.random.seed(42)  # Set the seed for reproducibility

# Perform bootstrapping and calculate means and confidence intervals
results = []

for (model, prompt, dataset), group in df.groupby(['model', 'prompt', 'dataset']):
    bootstrapped_means = []
    for _ in range(n_bootstraps):
        resampled = group['correct_answer'].sample(n=len(group), replace=True)
        bootstrapped_means.append(resampled.mean())
    
    mean = np.mean(bootstrapped_means)
    lower = np.percentile(bootstrapped_means, (1 - confidence_level) / 2 * 100)
    upper = np.percentile(bootstrapped_means, (1 + confidence_level) / 2 * 100)
    
    results.append({
        'dataset': dataset,
        'model': model,
        'prompt': prompt,
        'mean': mean,
        'ci_lower': lower,
        'ci_upper': upper
    })

# Convert the results to a DataFrame and pivot
results_df = pd.DataFrame(results)
mean_pivot = results_df.pivot_table(values='mean', index='model', columns='prompt')
ci_pivot_lower = results_df.pivot_table(values='ci_lower', index='model', columns='prompt')
ci_pivot_upper = results_df.pivot_table(values='ci_upper', index='model', columns='prompt')

In [289]:
results_df

,dataset,model,prompt,mean,ci_lower,ci_upper
0,commonsense_qa,command-xlarge-nightly,None_None,0.5139,0.3636,0.6970
1,med_qa,command-xlarge-nightly,None_None,0.1840,0.0606,0.3030
2,medmc_qa,command-xlarge-nightly,None_None,0.2764,0.1212,0.4545
3,open_book_qa,command-xlarge-nightly,None_None,0.5718,0.3939,0.7273
4,strategy_qa,command-xlarge-nightly,None_None,0.5385,0.3636,0.6970
...,...,...,...,...,...,...
355,med_qa,text-davinci-003,zhou-01-ins_None,0.3573,0.2121,0.5152
356,medmc_qa,text-davinci-003,zhou-01-ins_None,0.3257,0.1811,0.4848
357,open_book_qa,text-davinci-003,zhou-01-ins_None,0.8153,0.6970,0.9394
358,strategy_qa,text-davinci-003,zhou-01-ins_None,0.6358,0.4848,0.7879


In [262]:
mean_pivot["None_None"]
ci_pivot_lower["None_None"]
ci_pivot_upper["None_None"]

model
command-xlarge-nightly    0.449747
flan-T5-xxl               0.620707
gpt-3.5-turbo             0.711854
gpt-4                     0.792465
text-davinci-002          0.566076
text-davinci-003          0.594848
Name: None_None, dtype: float64

In [285]:
0.72*198

142.56

In [277]:
pd.options.display.float_format = "{:.4f}".format

In [278]:
import pandas as pd

# Combine the DataFrames
final_df = mean_pivot.copy()
for col in mean_pivot.columns:
    final_df[col] = mean_pivot[col].round(4).astype(str) + " ({\\scriptsize " + ci_pivot_lower[col].round(4).astype(str) + ", " + ci_pivot_upper[col].round(4).astype(str) + "})"

# Apply custom formatting (use LaTeX format for a scientific paper)
latex_table = final_df.to_latex(column_format="l" + "c" * len(final_df.columns), float_format="{:.2f}".format, escape=False)

print(latex_table)


\begin{tabular}{lcccccccccc}
\toprule
prompt &                              None_None &                         None_kojima-01 &                           None_zhou-01 &                             qa-10_None &                             qa-12_None &                             qa-13_None &                             qa-16_None &                             qa-17_None &                           refl-01_None &                       zhou-01-ins_None \\
model                  &                                        &                                        &                                        &                                        &                                        &                                        &                                        &                                        &                                        &                                        \\
\midrule
command-xlarge-nightly &  0.4497 ({\scriptsize 0.3838, 0.5152}) &  0.4239 ({\scriptsize 0.3535, 0

/tmp/ipykernel_1390/2982337073.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = final_df.to_latex(column_format="l" + "c" * len(final_df.columns), float_format="{:.2f}".format, escape=False)


In [279]:
print(latex_table)

\begin{tabular}{lcccccccccc}
\toprule
prompt &                              None_None &                         None_kojima-01 &                           None_zhou-01 &                             qa-10_None &                             qa-12_None &                             qa-13_None &                             qa-16_None &                             qa-17_None &                           refl-01_None &                       zhou-01-ins_None \\
model                  &                                        &                                        &                                        &                                        &                                        &                                        &                                        &                                        &                                        &                                        \\
\midrule
command-xlarge-nightly &  0.4497 ({\scriptsize 0.3838, 0.5152}) &  0.4239 ({\scriptsize 0.3535, 0

$$
\begin{tabular}{lcc}
\toprule
Prompt &                p1 &                p2 \\
Model &                   &                   \\
\midrule
A     &  0.5 (0.25, 0.75) &  0.5 (0.25, 0.75) \\
B     &   0.75 (0.5, 1.0) &  0.5 (0.25, 0.75) \\
\bottomrule
\end{tabular}
$$

In [ ]:
from cot.stats import evaluation_as_table

In [ ]:
eval = coll.evaluate()
table = evaluation_as_table(eval)
table

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

/home/kon/work/ThoughtSource/libs/cot/cot/stats.py:406: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[dataset, (instruction + "_" + cot_trigger, model)] = v


In [29]:
def table_function_new(eval:dict):
    import pandas as pd
    eval_dict = pd.json_normalize(eval).to_dict('records')[0]
    eval_list = list(eval_dict.keys())
    datasets = sorted(list(eval.keys()))

    models = []
    prompts = []
    for i in eval_list:
        # fast fix for chat gpt model:
        i = i.replace("gpt-3.5-turbo","gpt-3-5-turbo")
        dataset,split,metric,model,prompt = i.split(".")
        model = model.replace("gpt-3-5-turbo","gpt-3.5-turbo")
        if model not in models:
            models.append(model)
        if prompt not in prompts:
            prompts.append(prompt)
            
    models = sorted(models)

    if "None_None_None" in prompts: prompts.remove("None_None_None")

    # no instructions implemented yet
    # instructions = []
    cot_triggers = []
    for i in prompts:
        instruction, cot_trigger, _ = i.split("_")
        # if instruction not in instructions:
        #     instructions.append(instruction)

        # old: only cot_trigger
        # if cot_trigger not in cot_triggers:
        #     cot_triggers.append(cot_trigger)

        # new: instruction + cot_trigger
        if cot_trigger not in cot_triggers:
            cot_triggers.append(instruction + "_" + cot_trigger)

    cot_triggers = sorted(cot_triggers)

    cot_trigger_header = sorted(cot_triggers*len(models))
    model_header = models*len(cot_triggers)

    # Create a dictionary to store the data
    data_dict = {}
    for k,v in eval_dict.items():
        # fast fix for chat gpt model:
        k = k.replace("gpt-3.5-turbo","gpt-3-5-turbo")
        dataset,split,metric,model,prompt = k.split(".")
        model = model.replace("gpt-3-5-turbo","gpt-3.5-turbo")
        instruction, cot_trigger, _ = prompt.split("_")
        # old: only cot_trigger
        # df.loc[dataset, (cot_trigger, model)] = v

        # new: instruction + cot_trigger
        col_name = instruction + "_" + cot_trigger + "_" + model
        if dataset not in data_dict:
            data_dict[dataset] = {}
        data_dict[dataset][col_name] = v

    # Create a dataframe from the dictionary
    df = pd.DataFrame.from_dict(data_dict, orient='index')

    # Calculate the average row and add it to the dataframe
    df.loc['Average'] = df.mean()

    # Round the dataframe to 2 decimal places
    df = df.round(2)

    return df

In [41]:
df = table_function_new(eval)
df

,None_None_command-xlarge-nightly,None_kojima-01_command-xlarge-nightly,None_zhou-01_command-xlarge-nightly,qa-10_None_command-xlarge-nightly,qa-12_None_command-xlarge-nightly,qa-13_None_command-xlarge-nightly,qa-16_None_command-xlarge-nightly,qa-17_None_command-xlarge-nightly,refl-01_None_command-xlarge-nightly,zhou-01-ins_None_command-xlarge-nightly,...,None_None_text-davinci-003,None_kojima-01_text-davinci-003,None_zhou-01_text-davinci-003,qa-10_None_text-davinci-003,qa-12_None_text-davinci-003,qa-13_None_text-davinci-003,qa-16_None_text-davinci-003,qa-17_None_text-davinci-003,refl-01_None_text-davinci-003,zhou-01-ins_None_text-davinci-003
commonsense_qa,0.52,0.48,0.73,0.70,0.70,0.76,0.67,0.61,0.70,0.73,...,0.73,0.67,0.67,0.70,0.79,0.67,0.58,0.64,0.76,0.73
med_qa,0.18,0.30,0.27,0.27,0.27,0.24,0.33,0.30,0.21,0.24,...,0.30,0.33,0.39,0.36,0.30,0.39,0.36,0.24,0.33,0.36
medmc_qa,0.27,0.15,0.42,0.27,0.30,0.36,0.36,0.36,0.24,0.33,...,0.36,0.36,0.48,0.45,0.36,0.36,0.39,0.42,0.39,0.33
open_book_qa,0.58,0.42,0.61,0.58,0.58,0.58,0.55,0.61,0.61,0.64,...,0.64,0.58,0.79,0.73,0.79,0.61,0.64,0.73,0.64,0.82
strategy_qa,0.55,0.58,0.24,0.12,0.09,0.12,0.33,0.30,0.06,0.09,...,0.61,0.64,0.64,0.58,0.67,0.70,0.55,0.58,0.64,0.64
worldtree,0.61,0.61,0.76,0.73,0.82,0.85,0.85,0.64,0.79,0.82,...,0.94,0.91,0.82,0.82,0.91,0.85,0.82,0.91,0.91,0.88
Average,0.45,0.42,0.51,0.44,0.46,0.48,0.52,0.47,0.43,0.47,...,0.60,0.58,0.63,0.61,0.64,0.60,0.56,0.59,0.61,0.63


In [49]:
df = (df[df.index=="Average"])
#df.to_csv("Average_values_df.csv")
df

,None_None_command-xlarge-nightly,None_kojima-01_command-xlarge-nightly,None_zhou-01_command-xlarge-nightly,qa-10_None_command-xlarge-nightly,qa-12_None_command-xlarge-nightly,qa-13_None_command-xlarge-nightly,qa-16_None_command-xlarge-nightly,qa-17_None_command-xlarge-nightly,refl-01_None_command-xlarge-nightly,zhou-01-ins_None_command-xlarge-nightly,...,None_None_text-davinci-003,None_kojima-01_text-davinci-003,None_zhou-01_text-davinci-003,qa-10_None_text-davinci-003,qa-12_None_text-davinci-003,qa-13_None_text-davinci-003,qa-16_None_text-davinci-003,qa-17_None_text-davinci-003,refl-01_None_text-davinci-003,zhou-01-ins_None_text-davinci-003
Average,0.45,0.42,0.51,0.44,0.46,0.48,0.52,0.47,0.43,0.47,...,0.6,0.58,0.63,0.61,0.64,0.6,0.56,0.59,0.61,0.63


In [12]:
import json, re
import pandas as pd
def clean_column(df, col_name):
    pattern = r"^.?'(.?)'.?'(.?)'.?'(.?)'.?'(.?)'.*$"
    return(df[col_name].apply(lambda x: re.sub(pattern, r"'\3", x)[1:]))

def json_to_dataframe_3(json_data):
    df_data = []
    for category, data in json_data.items():
        for subset, questions in data.items():
            for question in questions:
                for generated_cot in question['generated_cot']:
                    row = {
                        'dataset': category,
                        'split': subset,
                        'id': question['id'],
                        'model': generated_cot['model'],
                        'instruction': generated_cot['instruction'],
                        'cot_trigger': generated_cot["cot_trigger"],
                        'generated_cot': generated_cot['cot'],
                        'correct_answer': generated_cot['answers'][0]['correct_answer'],
                    }
                    df_data.append(row)
    df = pd.DataFrame(df_data)
    # df["model"] = clean_column(df, "model")
    return df


In [13]:
df["model"][0]

"'name': 'command-xlarge-nightly', 'temperature': 0, 'max_tokens': 512}"

In [10]:
path = "/home/kon/work/ThoughtSource/libs/cot/cot/datasets/thoughtsource/thoughtsource_33_paper.json"


with open(path, "r") as read_file:
    data = json.load(read_file)

df = json_to_dataframe_3(data)

df

,dataset,split,id,model,instruction,cot_trigger,generated_cot,correct_answer
0,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'command-xlarge-nightly', 'temperature...",None,None,The main purpose of farmers is to supply food.,True
1,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'text-davinci-003', 'temperature': 0, ...",None,None,E) supply food,True
2,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'flan-T5-xxl', 'temperature': 0, 'max_...",None,None,E,True
3,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'gpt-3.5-turbo', 'temperature': 0, 'ma...",None,None,E) supply food,True
4,commonsense_qa,validation,9aff72f0c480c2b4edde45bd2e7e4870,"'name': 'text-davinci-002', 'temperature': 0, ...",None,None,\nE) supply food,True
...,...,...,...,...,...,...,...,...
11875,worldtree,test,worldtree_test_1659,"'name': 'flan-T5-xxl', 'temperature': 0, 'max_...",None,zhou-01,C is the dependent variable. The dependent var...,True
11876,worldtree,test,worldtree_test_1659,"'name': 'text-davinci-003', 'temperature': 0, ...",None,zhou-01,\n\nStep 1: The vertical axis on a graph is t...,False
11877,worldtree,test,worldtree_test_1659,"'name': 'text-davinci-002', 'temperature': 0, ...",None,zhou-01,\n\nThe first thing we need to do is identify...,True
11878,worldtree,test,worldtree_test_1659,"'name': 'command-xlarge-nightly', 'temperature...",None,zhou-01,\nThe y-axis is the vertical axis on a graph. ...,True
